In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
R = requests.get("https://it.ku.edu.tr/en/educational-technologies/classroom-search/")
SOUP = BeautifulSoup(R.content, 'html5lib')

In [3]:
URL_HEAD = "https://it.ku.edu.tr/en/classroom/"

URL_LIST = []

In [4]:
class_list = SOUP.findAll('a', attrs={'style':'color:black;'})
for class_title in class_list:
    
    class_name = class_title.text.lower().replace(" ", "-")
    
    if len(class_name.split("-")) > 2:
        continue
        
    URL_LIST.append(URL_HEAD + class_name + "/")

In [5]:
def create_tuple(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')
    
    ### 
    titles = soup.findAll('span', attrs = {'class':'title'})
    class_name = titles[1].text.strip()

    building = class_name.split()[0]
    floor = class_name.split()[1][0]
    room = class_name.split()[1][1:]

    class_name = building + '_' + floor + room
    ###
    
    ###
    mylists = soup.findAll('li')

    class_info = []
    for ls in mylists[:-1]: 
        if (ls.get('class') == ['col-md-12']):
            class_info.append(ls.text.strip())
            
    ###
    
    class_dict = {}
    for info in class_info:
        class_dict[info.split(":")[0].strip()] = info.split(":")[1].strip()
        

    ###
    has_window = True if class_dict["Window"] == "Yes" else False
    no_boards = int(class_dict["Number of Boards"]) if class_dict["Number of Boards"].isdigit() else 0
    CONNECTIONS = list(class_dict["Available Connections"].split(", ")) if len(class_dict["Available Connections"]) > 0 else []
    no_projector = int(class_dict["Projector"]) if class_dict["Projector"].isdigit() else 0
    has_lectureCapture = True if class_dict["Lecture Capturing System"] == "Yes" else False
    has_touchScreen = True if class_dict["Touch screen with Digital Pen"] == "Yes" else False
    has_docCamera = True if class_dict["Document Camera"] == "Yes" else False
    has_outlets = True if class_dict["Outlets For Students"] == "Yes" else False
    has_Camera = True if class_dict["Camera"] == "Yes" else False
    capacity = int(class_dict["Capacity"]) if class_dict["Capacity"].isdigit() else None
    ###
    
    ###
    class_tuple = [class_name, building, floor, room, capacity, has_window, has_Camera, has_docCamera, has_outlets, no_boards, no_projector, has_touchScreen, has_lectureCapture]
    ### 
    
    connection_tuple = []
    for connection in CONNECTIONS:
        connection_tuple.append([class_name, connection])

    
    return class_tuple, connection_tuple

In [6]:
CLASSES_DF = pd.DataFrame(columns = ['class_ID', 'building', 'floor', 
                                     'class_no', 'capacity', 'has_window', 
                                     'has_camera', 'has_docCamera', 'has_outlets', 
                                     'no_boards', 'no_projector', 'has_touchScreen', 'has_LectureCap'])

CONNECTIONS_DF = pd.DataFrame(columns = ['class_ID', 'connection_type'])

In [7]:
def populate_data(URL_list):
    
    for URL in URL_list:
        
        try:
            class_tuple, connection_tuples = create_tuple(URL)
            print("Class: " + class_tuple[0] + " is added.")
        except:
            print("Error: " + URL)
            continue
            
        if class_tuple[0] in list(CLASSES_DF['class_ID']):
            break
        
        CLASSES_DF.loc[len(CLASSES_DF)] = class_tuple
        
        for i in range(len(connection_tuples)):
            CONNECTIONS_DF.loc[len(CONNECTIONS_DF)] = connection_tuples[i]

In [8]:
populate_data(URL_LIST)

Class: SCI_Z42 is added.
Class: SNA_A43 is added.
Class: SNA_B242 is added.
Class: SNA_B149 is added.
Class: SNA_B220 is added.
Class: ENG_B50 is added.
Class: ENG_128 is added.
Class: ENG_Z21 is added.
Class: ENG_B19 is added.
Class: ELC_B08 is added.
Class: ENG_Z15 is added.
Class: SNA_B172 is added.
Class: SNA_B153 is added.
Class: SNA_B152 is added.
Class: SOS_B11 is added.
Class: SNA_B119 is added.
Class: ENG_Z50 is added.
Class: SOS_B08 is added.
Class: ENG_B30 is added.
Class: SOS_Z27 is added.
Class: SOS_Z21 is added.
Class: SOS_Z15 is added.
Class: SOS_Z14 is added.
Class: SOS_B35 is added.
Class: SOS_B30 is added.
Class: SOS_B29 is added.
Class: SOS_B28 is added.
Class: SOS_B21 is added.
Class: SOS_B16 is added.
Class: SOS_B10 is added.
Class: SOS_B07 is added.
Class: SOS_145 is added.
Class: SOS_144 is added.
Class: SOS_104 is added.
Class: SOS_103 is added.
Class: SNA_B184 is added.
Class: SNA_B173 is added.
Class: SNA_A52 is added.
Class: SNA_A51 is added.
Class: SNA_A42 i

In [9]:
CLASSES_DF.to_csv("class_data.csv", index = False)
CONNECTIONS_DF.to_csv("connection_data.csv", index = False)